In [26]:
target_list = ('crosswalk')

def coco_form():
    coco_format = dict(
        info=dict(
            year=2021, version="1.0", description="Pedestrian", contributor="AI Hub", url="https://aihub.or.kr/aidata/136", date_created='2021-11-26 02:39:00'
        ),
        licenses=[
            dict(id=0, name='', url='')
        ],
        categories=[
            dict(id=0, name='background', supercategory=None),
            dict(id=1, name='crosswalk', supercategory=None),
        ],
        images=[
        ],
        annotations=[
        ]
    )

    # with open('coco_format.json','w') as f:
    #     json.dump(coco_format, f, indent=4)

    return coco_format

In [27]:
coco_form()

{'info': {'year': 2021,
  'version': '1.0',
  'description': 'Pedestrian',
  'contributor': 'AI Hub',
  'url': 'https://aihub.or.kr/aidata/136',
  'date_created': '2021-11-26 02:39:00'},
 'licenses': [{'id': 0, 'name': '', 'url': ''}],
 'categories': [{'id': 0, 'name': 'background', 'supercategory': None},
  {'id': 1, 'name': 'crosswalk', 'supercategory': None}],
 'images': [],
 'annotations': []}

## Dataframe to COCO_Format
### 작업 순서
1. 중복없이 파일 이름만 빼온다
2. 파일 이름의 길이만큼 for 문을 돌리는데
- 각 파일 이름끼리 group 을 묶어서 좌표를 다 가져오고
- 그 정보를 cocoformat 형태에 넣는다.
- area 계산도 해서 넣어야 한다.

In [28]:
import pandas as pd

In [29]:
# file in list
# https://jaeyung1001.tistory.com/44
from ast import literal_eval
test_xml = pd.read_csv('crosswalk_filtering.csv')
test_xml

,root_folder,sub_folder,xml_name,file_name,occluded,classes,polygon,subclass
0,Surface_3,Surface_433,573_SM1022_IMT04.xml,MP_SEL_SUR_028645.jpg,0,alley,"1394.02,0.00;1385.27,19.95;1765.84,233.20;1834...",crosswalk
1,Surface_3,Surface_433,573_SM1022_IMT04.xml,MP_SEL_SUR_028648.jpg,0,alley,"0.00,449.00;182.62,419.83;455.92,371.72;641.19...",crosswalk
2,Surface_1,Surface_126,164_SM1004_IMT08.xml,MP_SEL_SUR_009638.jpg,0,roadway,"775.66,374.85;776.27,311.37;782.99,295.19;779....",crosswalk
3,Surface_1,Surface_126,164_SM1004_IMT08.xml,MP_SEL_SUR_009638.jpg,0,roadway,"695.70,362.10;693.64,302.85;691.53,75.03;686.6...",crosswalk
4,Surface_1,Surface_126,164_SM1004_IMT08.xml,MP_SEL_SUR_009641.jpg,0,roadway,"349.48,166.61;321.80,0.00;728.18,0.00;678.35,3...",crosswalk
...,...,...,...,...,...,...,...,...
4882,Surface_3,Surface_450,549_SM1023_01.xml,MP_SEL_SUR_027070.jpg,0,roadway,"560.30,65.50;624.42,54.86;627.30,70.80;628.40,...",crosswalk
4883,Surface_3,Surface_450,549_SM1023_01.xml,MP_SEL_SUR_027075.jpg,0,roadway,"1686.82,40.24;1836.95,52.78;1920.00,55.79;1919...",crosswalk
4884,Surface_3,Surface_450,549_SM1023_01.xml,MP_SEL_SUR_027075.jpg,0,roadway,"1060.20,189.70;1147.30,158.60;1146.12,198.90;1...",crosswalk
4885,Surface_3,Surface_450,549_SM1023_01.xml,MP_SEL_SUR_027075.jpg,0,roadway,"1173.29,129.70;1219.60,140.07;1220.90,238.40;1...",crosswalk


In [30]:
target_list = ('crosswalk')

mapped_classes = test_xml.replace(target_list, 1)
mapped_classes

,root_folder,sub_folder,xml_name,file_name,occluded,classes,polygon,subclass
0,Surface_3,Surface_433,573_SM1022_IMT04.xml,MP_SEL_SUR_028645.jpg,0,alley,"1394.02,0.00;1385.27,19.95;1765.84,233.20;1834...",1
1,Surface_3,Surface_433,573_SM1022_IMT04.xml,MP_SEL_SUR_028648.jpg,0,alley,"0.00,449.00;182.62,419.83;455.92,371.72;641.19...",1
2,Surface_1,Surface_126,164_SM1004_IMT08.xml,MP_SEL_SUR_009638.jpg,0,roadway,"775.66,374.85;776.27,311.37;782.99,295.19;779....",1
3,Surface_1,Surface_126,164_SM1004_IMT08.xml,MP_SEL_SUR_009638.jpg,0,roadway,"695.70,362.10;693.64,302.85;691.53,75.03;686.6...",1
4,Surface_1,Surface_126,164_SM1004_IMT08.xml,MP_SEL_SUR_009641.jpg,0,roadway,"349.48,166.61;321.80,0.00;728.18,0.00;678.35,3...",1
...,...,...,...,...,...,...,...,...
4882,Surface_3,Surface_450,549_SM1023_01.xml,MP_SEL_SUR_027070.jpg,0,roadway,"560.30,65.50;624.42,54.86;627.30,70.80;628.40,...",1
4883,Surface_3,Surface_450,549_SM1023_01.xml,MP_SEL_SUR_027075.jpg,0,roadway,"1686.82,40.24;1836.95,52.78;1920.00,55.79;1919...",1
4884,Surface_3,Surface_450,549_SM1023_01.xml,MP_SEL_SUR_027075.jpg,0,roadway,"1060.20,189.70;1147.30,158.60;1146.12,198.90;1...",1
4885,Surface_3,Surface_450,549_SM1023_01.xml,MP_SEL_SUR_027075.jpg,0,roadway,"1173.29,129.70;1219.60,140.07;1220.90,238.40;1...",1


In [31]:
# define fields
# images
LICENCE = 0
URL = None
DATA_CAPTURED = None
WIDTH = 1920
HEIGHT = 1080
BBOX = []
AREA = 0
ISCROWD = 0

file_list = mapped_classes["file_name"].tolist()
set_file_list = set(file_list)
print(f'total #bbox : {len(file_list)},\timages: {len(set_file_list)}')

total #bbox : 4887,	images: 3478


In [ ]:
set_file_list = sorted(set_file_list)
set_file_list

In [38]:
from tqdm import tqdm
image_id_index = 0
coco = coco_form()
for image in tqdm(set_file_list):
    condition = mapped_classes['file_name'] == image
    coco['images'].append(
        dict(license=LICENCE, url=URL, file_name=image, height=HEIGHT, width=WIDTH, data_captured=DATA_CAPTURED, id=image_id_index)
        )

    box_id_index = 0
    for polygon in mapped_classes[condition].to_dict('records'):
        SEGMENTATION = list()
        
        polygons = polygon['polygon'].split(';')

        for polygon_xy in polygons:
            X, Y = polygon_xy.split(',')
            SEGMENTATION.append(int(float(X)))
            SEGMENTATION.append(int(float(Y)))
        # SEGMENTATION.append(SEGMENTATION[0])
        # SEGMENTATION.append(SEGMENTATION[1])

        coco['annotations'].append(
            dict(id=box_id_index, image_id=image_id_index, category_id=polygon['subclass'], bbox=BBOX, 
                area=AREA, segmentation=[SEGMENTATION], iscrowd=ISCROWD)
            )
        box_id_index+=1
    image_id_index+=1

100%|██████████| 3478/3478 [00:04<00:00, 822.18it/s]


In [39]:
# 잘 들어갔는지 검증
print(f'total #bbox : {len(coco["annotations"])},\timages: {len(coco["images"])}')

total #bbox : 4887,	images: 3478


In [40]:
import json
with open('crosswalk_filtering_open.json', 'w') as f:
    json.dump(coco,f, indent=4)